# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [7]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [8]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

5. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

6. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

7. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOooBff_WtnL8p_jmZ0yFi73s6XR4y0rnMkpqX-l
KBG3WlJDfuhxd)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

8. [The Batman (2022) Locations](https://www.latlong.net/location/the-batman-2022-locations-427)
Source: Latitude and Longitude Finder

The Batman (2022) was filmed in Liverpool and London, England, UK. Filming also took place at Cardington Studios 
and Warner Bros. Leavesden Studios.

9. [Category:Film Locations - Batman Wiki - Fandom](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

A · Arctic World · Arkham Asylum (Nolanverse) · Arkham Asylum (Schumacherverse) · Arkham State Hospital ...

Out: None

...

[Step 2: Duration 16.76 seconds| Input tokens: 13,029 | Output tokens: 3,105]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract supercar factory locations from the search results                                                     
  supercar_locations = [                                                                                           
      ("Ferrari", "Maranello", "Italy", "43.8018° N", "10.9914° E"),                                               
      ("Lamborghini", "Sant'Agata Bolognese", "Italy", "44.5055° N", "11.1865° E"),                                
      ("Pagani", "San Cesario sul Panaro", "Italy", "44.6948° N", "10.6325° E"),                                   
      ("Ducati", "Bologna", "Italy", "44.4949° N", "11.3426° E"),                                                  
      ("SSC North America", "Richland", "Washington", "46.1636° N", "119.2750° W"),                                
      ("Koenigsegg", "Angered", "Sweden", "59.3750° N", "13.4683° E"),                                             
      ("Bugatti", "Molsheim", "France", "48.6922° N", "7.7483° E"),                                                
      ("Porsche", "Leinfelden-Echterdingen", "Germany", "48.7758° N", "9.1829° E"),                                
      ("Aston Martin", "Gaydon", "England", "52.3774° N", "1.3151° W"),                                            
      ("McLaren", "Woking", "England", "51.3774° N", "-0.5714° W")                                                 
  ]                                                                                                                
                                                                                                                   
  # Function to convert coordinates to float                                                                       
  def convert_coords(coords):                                                                                      
      lat = float(coords[0].split('°')[0]) * (-1 if 'S' in coords[0] else 1)                                       
      lon = float(coords[1].split('°')[0]) * (-1 if 'W[38;2;230;219;11

Execution logs:
Batman Filming Locations:
                         Location        City        Country  Latitude  \
0             Necropolis Cemetery     Glasgow       Scotland   55.8642   
1               St. George's Hall   Liverpool        England   53.4093   
2                   Wollaton Hall  Nottingham        England   52.9522   
3      Carnegie Mellon University  Pittsburgh   Pennsylvania   40.4432   
4  Warner Bros. Leavesden Studios   Leavesden  Hertfordshire   51.7397   
5                  Kirkeby Center    Westwood    Los Angeles   34.0736   

   Longitude  Travel Time to Gotham  
0    -4.2518                   8.60  
1    -2.9915                   8.81  
2    -1.1553                   9.00  
3   -79.9432                   1.74  
4    -0.3871                   9.13  
5  -118.3989                   6.79  

Supercar Factories with Matching Travel Times:
Empty DataFrame
Columns: [Location, City, Country, Latitude, Longitude, Travel Time to Gotham]
Index: []

Final Result:
                                                  Location        City  \
Batman Filming Locations 0             Necropolis Cemetery     Glasgow   
                         1               St. George's Hall   Liverpool   
                         2                   Wollaton Hall  Nottingham   
                         3      Carnegie Mellon University  Pittsburgh   
                         4  Warner Bros. Leavesden Studios   Leavesden   
                         5                  Kirkeby Center    Westwood   

                                  Country  Latitude  Longitude  \
Batman Filming Locations 0       Scotland   55.8642    -4.2518   
                         1        England   53.4093    -2.9915   
                         2        England   52.9522    -1.1553   
                         3   Pennsylvania   40.4432   -79.9432   
                         4  Hertfordshire   51.7397    -0.3871   
                         5    Los Angeles   34.0736  -118.3989   

                            Travel Time to Gotham  
Batman Filming Locations 0                   8.60  
                         1                   8.81  
                         2                   9.00  
                         3                   1.74  
                         4                   9.13  
                         5                   6.79  

Out - Final answer:                                                   Location        City  \
Batman Filming Locations 0             Necropolis Cemetery     Glasgow   
                         1               St. George's Hall   Liverpool   
                         2                   Wollaton Hall  Nottingham   
                         3      Carnegie Mellon University  Pittsburgh   
                         4  Warner Bros. Leavesden Studios   Leavesden   
                         5                  Kirkeby Center    Westwood   

                                  Country  Latitude  Longitude  \
Batman Filming Locations 0       Scotland   55.8642    -4.2518   
                         1        England   53.4093    -2.9915   
                         2        England   52.9522    -1.1553   
                         3   Pennsylvania   40.4432   -79.9432   
                         4  Hertfordshire   51.7397    -0.3871   
                         5    Los Angeles   34.0736  -118.3989   

                            Travel Time to Gotham  
Batman Filming Locations 0                   8.60  
                         1                   8.81  
                         2                   9.00  
                         3                   1.74  
                         4                   9.13  
                         5                   6.79  

[Step 3: Duration 12.56 seconds| Input tokens: 22,880 | Output tokens: 4,002]

In [9]:
result

Location        City  \
Batman Filming Locations 0             Necropolis Cemetery     Glasgow   
                         1               St. George's Hall   Liverpool   
                         2                   Wollaton Hall  Nottingham   
                         3      Carnegie Mellon University  Pittsburgh   
                         4  Warner Bros. Leavesden Studios   Leavesden   
                         5                  Kirkeby Center    Westwood   

                                  Country  Latitude  Longitude  \
Batman Filming Locations 0       Scotland   55.8642    -4.2518   
                         1        England   53.4093    -2.9915   
                         2        England   52.9522    -1.1553   
                         3   Pennsylvania   40.4432   -79.9432   
                         4  Hertfordshire   51.7397    -0.3871   
                         5    Los Angeles   34.0736  -118.3989   

                            Travel Time to Gotham  
Batman Filming Locations 0                   8.60  
                         1                   8.81  
                         2                   9.00  
                         3                   1.74  
                         4                   9.13  
                         5                   6.79

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [10]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Perform a web search to find a comprehensive list of supercar factories worldwide.
4. For each supercar factory, visit the source URL to confirm the coordinates.
5. Calculate the cargo plane transfer time from Gotham to each Batman filming location.
6. Calculate the cargo plane transfer time from Gotham to each supercar factory.
7. Organize the collected data into a pandas dataframe with columns for location name, coordinates, distance from 
Gotham, and estimated transfer time.
8. Finalize the pandas dataframe to include both Batman filming locations and supercar factories with the same 
cargo plane transfer time.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman Filming Locations:
                         Location        City        Country  Latitude  \
0             Necropolis Cemetery     Glasgow       Scotland   55.8642   
1               St. George's Hall   Liverpool        England   53.4093   
2                   Wollaton Hall  Nottingham        England   52.9522   
3      Carnegie Mellon University  Pittsburgh   Pennsylvania   40.4432   
4  Warner Bros. Leavesden Studios   Leavesden  Hertfordshire   51.7397   
5                  Kirkeby Center    Westwood    Los Angeles   34.0736   

   Longitude  Travel Time to Gotham  
0    -4.2518                   8.60  
1    -2.9915                   8.81  
2    -1.1553                   9.00  
3   -79.9432                   1.74  
4    -0.3871                   9.13  
5  -118.3989                   6.79  

Supercar Factories with Matching Travel Times:
Empty DataFrame
Columns: [Location, City, Country, Latitude, Longitude, Travel Time to Gotham]
Index: []

Final Result:
                                                  Location        City  \
Batman Filming Locations 0             Necropolis Cemetery     Glasgow   
                         1               St. George's Hall   Liverpool   
                         2                   Wollaton Hall  Nottingham   
                         3      Carnegie Mellon University  Pittsburgh   
                         4  Warner Bros. Leavesden Studios   Leavesden   
                         5                  Kirkeby Center    Westwood   

                                  Country  Latitude  Longitude  \
Batman Filming Locations 0       Scotland   55.8642    -4.2518   
                         1        England   53.4093    -2.9915   
                         2        England   52.9522    -1.1553   
                         3   Pennsylvania   40.4432   -79.9432   
                         4  Hertfordshire   51.7397    -0.3871   
                         5    Los Angeles   34.0736  -118.3989   

                            Travel Time to Gotham  
Batman Filming Locations 0                   8.60  
                         1                   8.81  
                         2                   9.00  
                         3                   1.74  
                         4                   9.13  
                         5                   6.79

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
422 Client Error: Unprocessable Entity for url: 
https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: 
Root=1-67be0c29-5c18a74535c6c1944900e1f0;91fbd35e-946e-44e9-b0a5-fff5196b8a20)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 33881 `inputs` 
tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
Make sure 'conversational' task is supported by the model.

[Step 7: Duration 0.80 seconds| Input tokens: 122,039 | Output tokens: 3,954]

[Step 8: Duration 0.65 seconds| Input tokens: 148,546 | Output tokens: 4,638]

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67be0c2a-060f22a405cf4bc8100f0426;d699405a-900a-4015-bcd4-d5c5c58638ef)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 31976 `inputs` tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
Make sure 'conversational' task is supported by the model.

In [11]:
detailed_report

NameError: name 'detailed_report' is not defined

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [14]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [15]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [16]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [17]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [18]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                <think>
Okay, let's start breaking down the problem. The user wants a map with Batman filming locations and supercar 
factories, each showing the cargo plane travel time to Gotham. They need at least six points total.

First, I need to get the Batman filming locations. Since I don't have that data, I'll have to ask the web_agent to 
find them. The web_agent can search for reliable sources like IMDb or Wikipedia to list all the filming locations 
along with their coordinates. Once I have those, I'll need to calculate the travel time from each location to 
Gotham using the calculate_cargo_travel_time tool.

Next, I need supercar factories. Again, the web_agent can look up the top supercar manufacturers and their factory 
locations. For each factory, I'll get the coordinates and then compute the travel time to Gotham using the same 
tool.

Once I have both sets of locations, I need to combine them into a single dataset. The example code uses a DataFrame
with columns like 'centroid_lat', 'centroid_lon', 'name', and 'peak_hour' (which in this case will be replaced by 
'travel_time'). I'll structure the data accordingly.

Then, I'll use Plotly to create the scatter map, color-coding the points based on travel time. The color scale 
should be something sequential, like Magma in the example. After generating the map, I'll save it as 
'saved_map.png' and provide it as the final answer.

I need to make sure there are at least six points. If the Batman locations are fewer than six, I'll need more 
supercar factories to reach the total. Also, all coordinates must be accurate to ensure correct distance 
calculations. The cargo plane speed defaults to 750 km/h unless specified, but I should confirm if the web_agent 
finds a different standard speed.

Potential issues: Some filming locations might not have precise coordinates listed, so the web_agent might need to 
approximate based on cities or landmarks. Also, ensuring that the web_agent provides enough factories if Batman 
locations are insufficient. Once all data is collected and times calculated, plotting should follow the example 
code structure.
</think>

1. Task the web_agent to compile a list of all Batman movie filming locations worldwide, including their geographic
coordinates (latitude/longitude).  
2. Task the web_agent to identify factories of supercar manufacturers globally, including their geographic 
coordinates.  
3. For each Batman filming location, use the `calculate_cargo_travel_time` tool to compute travel time from Gotham 
(40.7128° N, 74.0060° W).  
4. For each supercar factory, use the `calculate_cargo_travel_time` tool to compute travel time from Gotham.  
5. Combine the Batman locations and supercar factories into a single dataset, ensuring at least 6 total points.  
6. Generate a spatial map using the provided code template, coloring points by travel time and saving to 
`saved_map.png`.  
7. Return the map via `final_answer`.  


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 [STEP 1]:                                                                                                         
**Thought:** First, I need to get the list of Batman filming locations with their coordinates. Since I don’t have  
this data, I’ll ask the web_agent to research this.                                                                
**Code:**                                                                                                          
```py                                                                                                              
batman_task = "Find all filming locations for Batman movies (e.g., The Dark Knight Trilogy, Batman v Superman)     
worldwide. For each location, provide the name and exact latitude/longitude coordinates."                          
web_agent(task=batman_task)                                                                                        
```                                                                                                                
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_task = "Find all filming locations for Batman movies (e.g., The Dark Knight Trilogy, Batman v Superman)   
  worldwide. For each location, provide the name and exact latitude/longitude coordinates."                        
  web_agent(task=batman_task)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_response = web_agent(task="List Batman filming locations with coordinates")                               
  print("Batman response:", batman_response)  # Check the structure                                                
                                                                                                                   
  # Check if response is a string and parse it as JSON if possible                                                 
  import json                                                                                                      
  if isinstance(batman_response, str):                                                                             
      try:                                                                                                         
          batman_data = json.loads(batman_response)                                                                
      except json.JSONDecodeError:                                                                                 
          batman_data = {"final_results": []}                                                                      
  else:                                                                                                            
      batman_data = batman_response                                                                                
                                                                                                                   
  batman_locations = []                                                                                            
  for loc in batman_data.get("final_results", []):                                                                 
      if "name" in loc and "coordinates" in loc:                                                                   
          batman_locations.append({                                                                                
              "name": loc["name"],                                                                                 
              "lat": loc["coordinates"][0],                                                                        
              "lon": loc["coordinates"][1]                                                                         
          })                                                                                                       
  print("Processed Batman locations:", batman_locations)                                                           
                                                                                                                   
  # Get Batman locations with error handling                                                                       
  batman_response = web_agent(task="Provide Batman filming locations with exact coordinates (latitude,             
  longitude)")                                                                                                     
  print("Raw Batman response:", batman_response)  # Debugging                                                      
                                                                                                                   
  # Handle response format (string/dict)                                                                           
  if isinstance(batman_response, str):                                                                             
      import json                                                                                                  
      try:                                                                                                         
          batman_data = json.loads(batman_response)  # Attempt JSON parsing                                        
      except:                                            

Code execution failed at line 'search_results = web_search(query="Batman filming locations with coordinates")' due 
to: ValueError: {'error': 'Your account has run out of searches.'}

Code execution failed at line 'search_results = web_search(query="Batman filming locations with coordinates")' due 
to: ValueError: {'error': 'Your account has run out of searches.'}

[Step 6: Duration 105.33 seconds| Input tokens: 51,535 | Output tokens: 6,055]

KeyboardInterrupt: 

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [19]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)